In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")


153 documents
30 classes ['HATCHBACK', 'HONDA_HATCHBACK', 'HONDA_SEDAN', 'HONDA_SUV', 'HYUNDAI_HATCHBACK', 'HYUNDAI_SEDAN', 'HYUNDAI_SUV', 'MAHINDRA&TOYOTA_HATCHBACK', 'MAHINDRA_SEDAN', 'MAHINDRA_SUV', 'SEDAN', 'SUV', 'SUZUKI_HATCHBACK', 'SUZUKI_SEDAN', 'SUZUKI_SUV', 'TATA_HATCHBACK', 'TATA_SEDAN', 'TATA_SUV', 'TOYOTA_SEDAN', 'TOYOTA_SUV', 'companies', 'goodbye', 'greeting', 'hospital_search', 'options', 'other_companies', 'pharmacy_search', 'search_blood_pressure_by_patient_id', 'specific', 'thanks']
99 unique lemmatized words ["'s", ',', '123', '456', '6', 'a', 'about', 'all', 'any', 'anyone', 'are', 'awesome', 'be', 'brand', 'bye', 'can', 'car', 'chatting', 'choose', 'comapnies', 'company', 'condition', 'could', 'data', 'day', 'detail', 'do', 'find', 'for', 'good', 'goodbye', 'hatchback', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'honda', "honda'suvs", 'hospital', 'how', 'hyundai', "hyundai'suvs", 'i', 'in', 'indian', 'is', 'know', 'later', 'list', 'locate', 'looki

<ipython-input-3-9e01286392d0>:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


31/31 [==============================] - 1s 2ms/step - loss: 3.4451 - accuracy: 0.0261
Epoch 2/200
31/31 [==============================] - 0s 2ms/step - loss: 3.3512 - accuracy: 0.0458
Epoch 3/200
31/31 [==============================] - 0s 2ms/step - loss: 3.2684 - accuracy: 0.0980
Epoch 4/200
31/31 [==============================] - 0s 2ms/step - loss: 3.2004 - accuracy: 0.0784
Epoch 5/200
31/31 [==============================] - 0s 2ms/step - loss: 3.0425 - accuracy: 0.1569
Epoch 6/200
31/31 [==============================] - 0s 2ms/step - loss: 2.9337 - accuracy: 0.1765
Epoch 7/200
31/31 [==============================] - 0s 2ms/step - loss: 2.6714 - accuracy: 0.2549
Epoch 8/200
31/31 [==============================] - 0s 2ms/step - loss: 2.6211 - accuracy: 0.2092
Epoch 9/200
31/31 [==============================] - 0s 2ms/step - loss: 2.3820 - accuracy: 0.2876
Epoch 10/200
31/31 [==============================] - 0s 2ms/step - loss: 2.2535 - accuracy: 0.3203
Epoch 11/200
31/31 [=

31/31 [==============================] - 0s 2ms/step - loss: 0.2015 - accuracy: 0.9412
Epoch 84/200
31/31 [==============================] - 0s 2ms/step - loss: 0.2543 - accuracy: 0.9281
Epoch 85/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1779 - accuracy: 0.9477
Epoch 86/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1898 - accuracy: 0.9477
Epoch 87/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1731 - accuracy: 0.9281
Epoch 88/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1842 - accuracy: 0.9281
Epoch 89/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1818 - accuracy: 0.9412
Epoch 90/200
31/31 [==============================] - 0s 2ms/step - loss: 0.3087 - accuracy: 0.9150
Epoch 91/200
31/31 [==============================] - 0s 2ms/step - loss: 0.2530 - accuracy: 0.9281
Epoch 92/200
31/31 [==============================] - 0s 2ms/step - loss: 0.2322 - accuracy: 0.9085
Epoch 93/200


31/31 [==============================] - 0s 2ms/step - loss: 0.1322 - accuracy: 0.9673
Epoch 165/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1550 - accuracy: 0.9412
Epoch 166/200
31/31 [==============================] - 0s 2ms/step - loss: 0.2071 - accuracy: 0.9477
Epoch 167/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1391 - accuracy: 0.9608
Epoch 168/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1497 - accuracy: 0.9542
Epoch 169/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1333 - accuracy: 0.9739
Epoch 170/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1905 - accuracy: 0.9412
Epoch 171/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1427 - accuracy: 0.9477
Epoch 172/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1331 - accuracy: 0.9542
Epoch 173/200
31/31 [==============================] - 0s 2ms/step - loss: 0.1337 - accuracy: 0.9542
Epoc